In [1]:
#!ls "/content/drive/My Drive"


In [2]:
#!pip install gguf
#!git clone https://github.com/ggerganov/llama.cpp.git
#!pip install -r /content/llama.cpp/requirements.txt


In [3]:
import os

# Define paths
#merged_model_path = "/content/drive/MyDrive/merged_model_fp16"
#output_path = "/content/drive/MyDrive/merged_model_gguf"
#output_file = "model_fp16.gguf"

# Create output directory if it doesn't exist
#os.makedirs(output_path, exist_ok=True)

# Run the conversion script with FP16 settings
#!python /content/llama.cpp/convert.py {merged_model_path} \
 #   --outfile {os.path.join(output_path, output_file)} \
  #  --outtype f16


In [4]:
import os
print(os.listdir("/content/"))


['.config', 'drive', 'sample_data']


In [5]:
#!ls /content/llama.cpp


In [6]:
pip install unsloth

In [7]:
!pip install rouge-score nltk rapidfuzz scikit-learn


  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached rapidfuzz-3.12.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached rapidfuzz-3.12.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c5bf9f4e6aaf962900f264c80d83112513c30019d4b5d5160dcb34144db83afa
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [8]:
!pip install rouge

In [9]:
print(df_eval.columns)


NameError: name 'df_eval' is not defined

In [10]:
import nltk
import shutil
import pandas as pd
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from peft import PeftModel
from rapidfuzz.distance import Levenshtein
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 🔹 Step 1: Fix NLTK Issues (Ensure No Errors)
nltk.data.path.append('/usr/local/nltk_data')
shutil.rmtree('/root/nltk_data/tokenizers/punkt', ignore_errors=True)
nltk.download('punkt')

# 🔹 Step 2: Load Model & Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/content/drive/MyDrive/merged_model_fp16"  # Update with actual model path

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load fine-tuned model
model = FastLanguageModel.from_pretrained(model_path, load_in_4bit=True)[0]
model.to(device)

FastLanguageModel.for_inference(model)  # Optimize for inference
print("✅ Merged fine-tuned model loaded successfully!")

# 🔹 Step 3: Load Evaluation Data
eval_file = "/content/eval_data_v4.csv"  # Update with actual evaluation file path
df_eval = pd.read_csv(eval_file)

# 🔹 Step 4: MongoDB Query Generation Function
def generate_mongo_query(db_id, schema_info, nl_query):
    """ Generate MongoDB query using the fine-tuned model """
    prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context.
    Write a response that appropriately completes the request.

    ### Instruction:
    Generate a MongoDB query from the given database schema and natural language request.

    ### Input:
    Database ID: {db_id}
    Schema: {schema_info}
    Query: {nl_query}

    ### Response:
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=150)
    generated_query = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_query.split("### Response:")[-1].strip()

# 🔹 Step 5: Evaluation Metrics (No NLTK Errors!)
def compute_metrics(true_query, generated_query):
    """Compute similarity metrics between true and generated MongoDB queries"""

    # Levenshtein Distance (Lower = More Similar)
    lev_dist = Levenshtein.distance(true_query, generated_query)

    # Cosine Similarity (Higher = More Similar)
    vectorizer = CountVectorizer().fit_transform([true_query, generated_query])
    vectors = vectorizer.toarray()
    cos_sim = cosine_similarity([vectors[0]], [vectors[1]])[0][0]

    return {
        "Levenshtein Distance": lev_dist,
        "Cosine Similarity": cos_sim
    }

# 🔹 Step 6: Run Evaluation
all_metrics = []

for index, row in df_eval.iterrows():
    db_id = row["database_id"]  # Fixed column name
    schema_info = row["schema"]
    true_query = row["mongo_query"]
    nl_query = row["natural_language_query"]

    # Generate MongoDB query
    generated_query = generate_mongo_query(db_id, schema_info, nl_query)

    # Compute evaluation metrics
    metrics = compute_metrics(true_query, generated_query)
    all_metrics.append(metrics)

# 🔹 Step 7: Save & Display Results
df_results = pd.DataFrame(all_metrics)
print(df_results.head())

# Save results to a CSV file
df_results.to_csv("/content/evaluation_results.csv", index=False)
print("✅ Evaluation completed and saved as 'evaluation_results.csv'.")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Merged fine-tuned model loaded successfully!
   Levenshtein Distance  Cosine Similarity
0                   556           0.240772
1                   604           0.000000
2                   148           0.664364
3                   148           0.664364
4                   143           0.664364
✅ Evaluation completed and saved as 'evaluation_results.csv'.


In [16]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import DPOTrainer

# 🔹 Load Dataset
df = pd.read_csv("/content/train_dpo_data_1600.csv")
df.rename(columns={"corret_mongo_query": "correct_mongo_query"}, inplace=True)

def format_prompt(nl_query, schema):
    return f"""### Instruction:
Generate a MongoDB query from the given database schema and natural language request.

### Input:
Database Schema: {schema}
Query: {nl_query}

### Response:
"""

df["prompt"] = df.apply(lambda row: format_prompt(row["natural_language_query"], row["schema"]), axis=1)
dpo_dataset = Dataset.from_pandas(df[["prompt", "correct_mongo_query", "incorrect_mongo_query"]])

# 🔹 Fix Column Names for DPO
dpo_dataset = dpo_dataset.rename_columns({
    "correct_mongo_query": "chosen",
    "incorrect_mongo_query": "rejected"
})

# 🔹 Load Model with 4-bit Quantization
model_name = "/content/drive/MyDrive/merged_model_fp16"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

# 🔹 Attach LoRA Adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 🔹 Define `max_seq_length` in Tokenizer
tokenizer.model_max_length = 512

# 🔹 Tokenization Function
def tokenize_fn(samples):
    return tokenizer(samples["prompt"], truncation=True, padding="max_length", max_length=512)

dpo_dataset = dpo_dataset.map(tokenize_fn, batched=True)

# 🔹 Training Arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    output_dir="./dpo_finetuned_model",
    optim="adamw_torch",
    report_to="none"
)

# 🔹 DPOTrainer
trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=dpo_dataset,
    tokenizer=tokenizer,
    beta=0.1,
)

trainer.train()


trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Extracting prompt in train dataset (num_proc=2):   0%|          | 0/1600 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/1600 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1600 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,600 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 300
 "-____-"     Number of trainable parameters = 1,081,344


AttributeError: 'Qwen2Model' object has no attribute 'max_seq_length'